การติดตั้ง

In [ ]:
# pip install -U torch sahi yolov5
import os
os.getcwd()

In [ ]:
from sahi.utils.yolov5 import download_yolov5s6_model
from sahi.model import Yolov5DetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from IPython.display import Image

In [ ]:
yolov5_model_path = r'C:\bankprp\Projects\Palm_detection\best_weight.pt'
download_yolov5s6_model(destination_path=yolov5_model_path)

In [ ]:
# ตรวจสอบ gpu
import torch
torch.cuda.get_device_name()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

detection_model = Yolov5DetectionModel(
    model_path = yolov5_model_path,
    confidence_threshold = 0.3,
    device = device
)

Sliced Inference with a YOLOv5 Model

In [ ]:
result = get_sliced_prediction(
    r"C:\bankprp\Projects\Palm_detection\Palm_2.tif",
    detection_model,
    slice_height = 1024,
    slice_width = 1024,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2
)

In [ ]:
result.to_coco_annotations()

In [ ]:
import numpy as np
row_column = []
for c in result.to_coco_annotations() :
    xmin, ymin, width, height = c['bbox']
    center_x = xmin + width/2
    center_y = ymin + height/2
    row_column.append((center_x, center_y))


Rasterio

In [ ]:
# Install GDAL and rasterio wheel from https://www.lfd.uci.edu/~gohlke/pythonlibs/ then pip3 install <filename>
import rasterio
import rasterio.features
import rasterio.warp
import csv

In [ ]:
with rasterio.open(r'C:\bankprp\Projects\Palm_detection\Palm_2.tif') as dataset :
    print(dataset.profile)
    aff = dataset.transform

In [ ]:
with open(r'C:\bankprp\Projects\Palm_detection\palm_coor.csv', 'w') as f :
    writer = csv.writer(f)
    writer.writerow(['Latitude', 'Longtitude'])
    for c in row_column:
        writer.writerow(aff*c)